
# 01 — Structures de données & Complexités (version **complète**)

**Objectifs du cours**
- Comprendre **en profondeur** les structures Python (`list`, `tuple`, `dict`, `set`, `deque`, `heapq`, etc.)  
- Maîtriser les **complexités** (temps & mémoire) et **cas limites** fréquents en entretien  
- Savoir **choisir la bonne structure** selon le **cas d'usage** (+ pièges à éviter)  
- Disposer d'**exemples concrets**, d'**exercices** et d'une **fiche mémo**

**Notation** : toutes les équations sont en bloc `$$ ... $$` pour un rendu parfait.



## 1) Modèle interne (mentale) à avoir

- `list` : **tableau dynamique contigu** en mémoire → accès index **O(1)**, mais insertion/suppression **au milieu** = **O(n)** (décalage des éléments).
- `tuple` : comme `list` mais **immutable** → hashable si éléments hashables (peut servir de clé de dict).
- `dict` / `set` : **table de hachage** → opérations moyennes **O(1)**, **pas d'ordre logique** (mais ordre d'insertion préservé depuis Py ≥ 3.7 pour `dict`).  
- `deque` : **double-ended queue** basée sur des **blocs** (linked) → **O(1)** append/pop aux **deux extrémités** ; accès index **O(n)**.
- `heapq` : **min-heap** (tas binaire) → `heappush`/`heappop` **O(log n)**, le plus petit élément en tête.

**Rappel complexités asymptotiques**  
$$ \text{O(1)} < \text{O}(\log n) < \text{O}(n) < \text{O}(n \log n) < \text{O}(n^2) $$



## 2) Tableau de complexités (détaillé)

| Structure | Accès | Recherche | Insertion | Suppression | Itération | Remarques |
|---|---:|---:|---:|---:|---:|---|
| `list` | O(1) | O(n) | O(n) milieu / O(1) fin | O(n) milieu / O(1) fin | O(n) | Tableau contigu + over-allocation |
| `tuple` | O(1) | O(n) | — | — | O(n) | Immutable, hashable |
| `dict` | O(1)* | O(1)* | O(1)* | O(1)* | O(n) | *Amorti, dépend du hash/colisions |
| `set` | — | O(1)* | O(1)* | O(1)* | O(n) | Ensemble (éléments uniques) |
| `deque` | O(n) | O(n) | O(1) extrémités | O(1) extrémités | O(n) | Idéal files/rolling windows |
| `heapq` | — | — | O(log n) | O(log n) | O(n) | Min-heap, accès direct non supporté |

**Pourquoi `list.insert(i, x)` est O(n) ?**  
Parce qu'il faut **décaler** en mémoire **tous les éléments à droite** de `i`.  



## 3) Démonstrations de coûts : insert/pop au milieu vs en fin

On mesure l'impact **amorti** : en fin `append/pop` ~O(1), **au milieu** ~O(n).


In [1]:

import time, random

def bench_insert_middle(n=20000, reps=3):
    ts=[]
    for _ in range(reps):
        lst=list(range(n))
        t0=time.perf_counter()
        lst.insert(n//2, -1)
        ts.append(time.perf_counter()-t0)
    return sum(ts)/len(ts)

def bench_append(n=20000, reps=500):
    ts=[]
    for _ in range(reps):
        lst=[]
        t0=time.perf_counter()
        lst.append(1)
        ts.append(time.perf_counter()-t0)
    return sum(ts)/len(ts)

mid = bench_insert_middle()
end = bench_append()
print(f"insert(milieu) temps moyen: {mid*1e6:.2f} µs")
print(f"append(fin)     temps moyen: {end*1e6:.4f} µs  (~O(1))")


insert(milieu) temps moyen: 117.13 µs
append(fin)     temps moyen: 0.1648 µs  (~O(1))



## 4) Dictionnaires & Ensembles : hashing, collisions, pièges

- **Hashing** : `dict` et `set` s'appuient sur `__hash__` et `__eq__` pour les clés.  
- **Clés valides** : **immutables** et **hashables** (ex : `str`, `int`, `tuple` d'objets hashables).  
- **Collisions** : gérées par **open addressing** (détails CPython) ; la complexité reste **O(1) amorti**.

**Pièges classiques**
- Utiliser une **liste** en clé → ❌ (non hashable) ; utiliser un **tuple** → ✅
- Clés mutables (ex : `list`), ou objets dont `__hash__` dépend d'attributs qui changent → **comportements bizarres**.


In [2]:

# Clé tuple correcte
prices = {("EURUSD", "2024-01-01"): 1.09}
print("clé tuple OK:", prices[("EURUSD","2024-01-01")])

# Clé liste -> TypeError
try:
    bad = {["EURUSD","2024-01-01"]: 1.09}  # noqa
except TypeError as e:
    print("Erreur attendue:", e)


clé tuple OK: 1.09
Erreur attendue: unhashable type: 'list'



## 5) `deque` pour fenêtres glissantes (rolling) en **O(1) amorti**

Exemple : moyenne mobile sur un flux (évite `sum` à chaque pas).


In [3]:

from collections import deque
def rolling_mean(iterable, n):
    d=deque(maxlen=n); s=0.0
    for x in iterable:
        if len(d)==n:
            s -= d[0]
        d.append(x); s += x
        yield s/len(d)

print(list(rolling_mean([1,2,3,4,5,6], 3)))


[1.0, 1.5, 2.0, 3.0, 4.0, 5.0]



## 6) `heapq` (min-heap) : file de priorité, top‑k, fusion de flux ordonnés

- `heappush(h, x)`, `heappop(h)` → **O(log n)**
- Obtenir le **top‑k** efficacement (au lieu de trier tout)  
- Fusionner **k listes triées** (`heapq.merge`) → utile pour consolider des flux triés par timestamp.


In [4]:

import heapq, random

# Top-k (k plus grands) avec min-heap de taille k
def topk(iterable, k=5):
    h=[]
    for x in iterable:
        if len(h)<k:
            heapq.heappush(h, x)
        else:
            if x>h[0]:
                heapq.heapreplace(h, x)  # pop+push O(log k)
    return sorted(h, reverse=True)

data=[random.randint(0,1000) for _ in range(1000)]
print("Top 5:", topk(data, 5))


Top 5: [999, 998, 997, 997, 995]


In [5]:

# Fusion de flux triés (tapez 'help(heapq.merge)')
import heapq
a=[1,4,9]; b=[2,3,10]; c=[5,6,7,8]
print("merge:", list(heapq.merge(a,b,c)))


merge: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]



## 7) `list` vs `deque` : quand choisir quoi ?

- **`list`** : accès aléatoire rapide, **parcours** et **manipulations en fin**.
- **`deque`** : **file** FIFO/LIFO, besoins d'**append/pop aux deux extrémités**, **sliding window**.
- Accès `deque[i]` → **O(n)** ; `list[i]` → **O(1)**.  
**Règle** : si tu fais souvent `pop(0)`/`insert(0, x)` → prends une **`deque`**.



## 8) Exercices (avec corrections)

### Exo 1 — Choix de structure
> Tu reçois un flux de 10 millions d'IDs (peut contenir des doublons). Tu dois renvoyer la **liste des IDs uniques** et pouvoir tester `id in ...` rapidement. **Que choisis-tu ? Pourquoi ?**

**Attendu** : `set` pour unicité + membership O(1). Si ordre d'arrivée requis → `dict` (clé=id) pour préserver l'ordre (Py ≥3.7) ou `OrderedDict`.

---

### Exo 2 — Rolling maximum en O(1) amorti
> Implémente `rolling_max(arr, k)` qui renvoie le maximum de chaque fenêtre de taille `k` **en O(1) amorti**.

**Indication** : Utilise une `deque` qui garde des indices et des valeurs **monotoniquement décroissantes**.


In [6]:

from collections import deque

def rolling_max(arr, k):
    q = deque()
    res = []
    for i, x in enumerate(arr):
        # Retire les indices qui sortent de la fenêtre
        if q and q[0] <= i-k:
            q.popleft()
        # Maintient la deque décroissante (valeurs)
        while q and arr[q[-1]] <= x:
            q.pop()
        q.append(i)
        if i >= k-1:
            res.append(arr[q[0]])
    return res

print(rolling_max([1,3,-1,-3,5,3,6,7], 3))  # [3,3,5,5,6,7]


[3, 3, 5, 5, 6, 7]



## 9) Pièges d'entretien & points à **recaser**

- **Pourquoi `insert/pop` milieu list = O(n)` ?** → déplacement des éléments contigus.
- **Pourquoi `dict`/`set` = O(1) amorti ?** → table de hachage, dépend des collisions.
- **Quand `deque` > `list` ?** → opérations extrémités fréquentes (FIFO, rolling).
- **Pourquoi `heapq` ?** → priorité, top‑k, fusion k flux triés **sans trier tout**.
- **Clé hashable** → utiliser **tuple**, éviter **list** mutable.



---
## 📌 Fiche mémo (révision express)

- `list`: accès O(1), `append/pop` fin O(1), **milieu O(n)**  
- `tuple`: immutable, hashable si contenu hashable  
- `dict`/`set`: **O(1) amorti**, clés hashables, attention aux collisions  
- `deque`: **O(1)** aux deux extrémités, **O(n)** accès index  
- `heapq`: `heappush`/`heappop` **O(log n)**, top‑k, merges triés  
- Choix structure = **pattern d'accès** + **contraintes perf/mémoire**
